## Select PWV data

- creation date : 2025-11-27
- last update : 2025-11-30
- use conda_py313

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from PWV00_parameters import *

## Read File

In [ ]:
DumpConfig()

In [ ]:
filename_in = atmfilename 

In [ ]:
path_in = "data"
filename_in = os.path.basename(filename_in).replace('.npy','.csv')
fullfilename_in  = os.path.join(path_in,filename_in)

In [ ]:
path_out = "data/pwv"
filename_out = "auxtel_PWV_data.csv"
print(filename_out)

In [ ]:
fullfilename_out  = os.path.join(path_out,filename_out)

In [ ]:
df_spec = pd.read_csv(fullfilename_in)

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)
df_spec = df_spec.sort_values(by='Time')

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
df_spec = df_spec[df_spec["FILTER"].isin(["empty","OG550_65mm_1"]) ]

In [ ]:
cut = getSelectionCut(df_spec)

In [ ]:
df_spec_sel = df_spec[cut]

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)

In [ ]:
list_all_filts = df_spec_sel["FILTER"].unique()
list_all_filts= sorted(list_all_filts )
colors = {filt: col for filt, col in zip(list_all_filts, ["r","b"])}
colors = {
    'empty': "blue", 
    'FELH0600': "purple",
    'OG550_65mm_1':"red",
}

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")


fig,axs = plt.subplots(1,1,figsize=(18,8))
ax  = axs
leg=ax.get_legend()

#df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)

for filt, group in df_spec_sel.groupby("FILTER"):
    ax.errorbar(
                    group["Time"],
                    group["PWV [mm]_x"],
                    yerr= group["PWV [mm]_err_x"],
                    fmt="o",
                    label=filt,
                    color=colors[filt],
                    ecolor="k",
                    capsize=1,
                    markersize=5
                    )


ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Precipitable water vapor measured by holo vs time (after selection cut)")
ax.tick_params(axis="x", rotation=45)
ax.legend(loc="upper right")
ax.grid()


In [ ]:
df_spec_sel_save = df_spec_sel[["nightObs","seq_num","DATE-OBS","Time","PWV [mm]_x","PWV [mm]_err_x","TARGET","FILTER","AIRMASS","CHI2_FIT","PSF_REG", "TRACE_R", "MEANFWHM"]]

In [ ]:
df_spec_sel_save

In [ ]:
df_spec_sel_save.to_csv(filename_out)